# Preprocesamiento y transformación

## Limpieza de datos y detección de valores faltantes

### Ejemplo 1

#### Descripción de los datos

- ST_NUM: Street number
- ST_NAME: Street name
- OWN_OCCUPIED: Is the residence owner occupied
- NUM_BEDROOMS: Number of bedroom

#### Tipos esperados

- ST_NUM: float or int… some sort of numeric type
- ST_NAME: string
- OWN_OCCUPIED: string… Y (“Yes”) or N (“No”)
- NUM_BEDROOMS: float or int, a numeric type

---

#### Valores faltantes "estándar"

In [12]:
# Importación de librerías.
import pandas as pd
import numpy as np

# Lee el dataset.
df = pd.read_csv('datasets/property_data.csv')

# Hecha un vistazo a las primeras filas.
print(df.head())

           PID  ST_NUM    ST_NAME OWN_OCCUPIED NUM_BEDROOMS NUM_BATH SQ_FT
0  100001000.0   104.0     PUTNAM            Y            3        1  1000
1  100002000.0   197.0  LEXINGTON            N            3      1.5    --
2  100003000.0     NaN  LEXINGTON            N          NaN        1   850
3  100004000.0   201.0   BERKELEY           12            1      NaN   700
4          NaN   203.0   BERKELEY            Y            3        2  1600


In [11]:
# Muestra valores de la columna.
print(df['ST_NUM'])

# Determina los nulos y no nulos.
print(df['ST_NUM'].isnull())

0    104.0
1    197.0
2      NaN
3    201.0
4    203.0
5    207.0
6      NaN
7    213.0
8    215.0
Name: ST_NUM, dtype: float64
0    False
1    False
2     True
3    False
4    False
5    False
6     True
7    False
8    False
Name: ST_NUM, dtype: bool


#### Valores faltantes "no estándar"

In [18]:
print(df['NUM_BEDROOMS'])
print(df['NUM_BEDROOMS'].isnull()) # Pandas no reconoce todos los valores 
                                   # que para nosotros son "faltantes".

0      3
1      3
2    NaN
3      1
4      3
5    NaN
6      2
7      1
8     na
Name: NUM_BEDROOMS, dtype: object
0    False
1    False
2     True
3    False
4    False
5     True
6    False
7    False
8    False
Name: NUM_BEDROOMS, dtype: bool


In [19]:
# Crea una lista de los tipos de valores faltantes.
missing_values = ['n/a', 'na', '--']
df = pd.read_csv("datasets/property_data.csv", na_values=missing_values)

In [20]:
# Vuelve a mostrar los valores de la columna para comprobar que los faltantes
# ya son detectados.
print(df['NUM_BEDROOMS'])
print(df['NUM_BEDROOMS'].isnull())

0    3.0
1    3.0
2    NaN
3    1.0
4    3.0
5    NaN
6    2.0
7    1.0
8    NaN
Name: NUM_BEDROOMS, dtype: float64
0    False
1    False
2     True
3    False
4    False
5     True
6    False
7    False
8     True
Name: NUM_BEDROOMS, dtype: bool


#### Valores con tipos "inesperados"

In [22]:
print(df['OWN_OCCUPIED'])
print(df['OWN_OCCUPIED'].isnull())

0      Y
1      N
2      N
3     12
4      Y
5      Y
6    NaN
7      Y
8      Y
Name: OWN_OCCUPIED, dtype: object
0    False
1    False
2    False
3    False
4    False
5    False
6     True
7    False
8    False
Name: OWN_OCCUPIED, dtype: bool


In [24]:
# Busca valores numéricos para una columna que solo permite cadenas,
# y convierte el valor a nulo.
cnt=0
for row in df['OWN_OCCUPIED']:
    try:
        int(row)
        df.loc[cnt, 'OWN_OCCUPIED']=np.nan
    except ValueError:
        pass
    cnt+=1

In [27]:
# Muestra cantidad de nulos para cada característica.
print(df.isnull().sum())

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    2
NUM_BEDROOMS    3
NUM_BATH        1
SQ_FT           2
dtype: int64


In [29]:
# Determina si hay un nulo en cualquier lugar.
print(df.isnull().values.any())

True


In [31]:
# Cuenta el total de nulos
print(df.isnull().sum().sum())

11


#### Reemplazo de datos faltantes

In [36]:
# Reemplaza faltantes con un número.
df['ST_NUM'].fillna(125, inplace=True)

# Reemplaza en una ubicación específica.
df.loc[2,'ST_NUM'] = 125

# Reemplaza con la mediana.
median = df['NUM_BEDROOMS'].median()
df['NUM_BEDROOMS'].fillna(median, inplace=True)

---

### Ejemplo 2. Cleaning Financial Time Series data

In [39]:
# Importa librerías requeridas.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats
from datetime import datetime
from functools import reduce
import datetime

In [78]:
# Carga todos los datasets
dow     = pd.read_csv('datasets/dow.csv')
unemp   = pd.read_csv('datasets/unemp.csv')
oil     = pd.read_csv('datasets/oil.csv')
hstarts = pd.read_csv('datasets/hstarts.csv')
cars    = pd.read_csv('datasets/cars.csv')
retail  = pd.read_csv('datasets/retail.csv')

#### Visualización de datasets

In [42]:
dow.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-06-11,26180.589844,26248.669922,25998.869141,26048.509766,26048.509766,253490000
1,2019-06-12,26040.300781,26082.099609,25958.660156,26004.830078,26004.830078,223690000
2,2019-06-13,26036.939453,26146.910156,25995.710938,26106.769531,26106.769531,213400000
3,2019-06-14,26076.359375,26162.279297,25988.089844,26089.609375,26089.609375,208020000
4,2019-06-17,26108.529297,26165.779297,26049.800781,26112.529297,26112.529297,199500000


In [43]:
dow.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
248,2020-06-04,26226.490234,26384.099609,26082.310547,26281.820313,26281.820313,372190000
249,2020-06-05,26836.800781,27338.300781,26836.800781,27110.980469,27110.980469,542310000
250,2020-06-08,27232.929688,27580.210938,27232.480469,27572.439453,27572.439453,430200000
251,2020-06-09,27447.369141,27447.369141,27151.060547,27272.300781,27272.300781,381430000
252,2020-06-10,27251.890625,27355.220703,26938.050781,26989.990234,26989.990234,448930000


In [50]:
# Crea una lista con los datasets.
dflist = [dow, unemp, oil, hstarts, cars, retail]

# Para cada dataset, muestra sus dimensiones.
for i, dfr in enumerate(dflist):
    print(dflist[i].shape)

(253, 7)
(869, 2)
(397, 2)
(736, 2)
(533, 2)
(340, 2)


In [52]:
# Muestra valores estadísticos de cada dataset.
for i, dfr in enumerate(dflist):
    print(dflist[i].describe())
    print()

               Open          High           Low         Close     Adj Close  \
count    253.000000    253.000000    253.000000    253.000000    253.000000   
mean   26376.747869  26550.543216  26185.806486  26375.729087  26375.729087   
std     2142.002560   2031.430287   2262.003739   2143.619409   2143.619409   
min    19028.359375  19121.009766  18213.650391  18591.929688  18591.929688   
25%    25678.169922  25941.250000  25440.390625  25717.740234  25717.740234   
50%    26836.800781  26983.449219  26733.330078  26820.250000  26820.250000   
75%    27831.230469  27897.279297  27676.970703  27783.039063  27783.039063   
max    29440.470703  29568.570313  29406.750000  29551.419922  29551.419922   

             Volume  
count  2.530000e+02  
mean   3.466130e+08  
std    1.958864e+08  
min    8.615000e+07  
25%    2.372200e+08  
50%    2.795400e+08  
75%    3.893900e+08  
max    2.190810e+09  

           UNRATE
count  869.000000
mean     5.746260
std      1.692264
min      2.500000

In [55]:
# Muestra cuales datasets tienen valores nulos.
for i, dfr in enumerate(dflist):
    print(dflist[i].isnull().sum().sum())

0
0
0
0
0
0


#### Limpieza de datos

In [80]:
dow = pd.read_csv('datasets/dow.csv')

# Elimina columnas innecesarias.
dow.drop(['Open', 'High', 'Low', 'Adj Close', 'Volume'], axis=1, inplace=True)

# Muestra algunos registros del dataset ya sin las columnas eliminadas.
dow.head()

,Date,Close
0,2019-06-11,26048.509766
1,2019-06-12,26004.830078
2,2019-06-13,26106.769531
3,2019-06-14,26089.609375
4,2019-06-17,26112.529297


In [81]:
dow = pd.read_csv('datasets/dow.csv')

# Renombra columnas a mayúsculas para que concuerden con los otros datasets.
dow.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'ADJ CLOSE', 'VOLUME']

# Muestra resultado final.
dow.head()

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME
0,2019-06-11,26180.589844,26248.669922,25998.869141,26048.509766,26048.509766,253490000
1,2019-06-12,26040.300781,26082.099609,25958.660156,26004.830078,26004.830078,223690000
2,2019-06-13,26036.939453,26146.910156,25995.710938,26106.769531,26106.769531,213400000
3,2019-06-14,26076.359375,26162.279297,25988.089844,26089.609375,26089.609375,208020000
4,2019-06-17,26108.529297,26165.779297,26049.800781,26112.529297,26112.529297,199500000


In [82]:
# Concatena dataframes en uno solo.
dfs = [dow, unemp, oil, hstarts, cars, retail]

# Realiza joins con la columna fecha y elimina valores nulos.
df = reduce(lambda left,right: pd.merge(left,right,on='DATE', how='outer'), dfs).dropna()
df.head(5)

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME,UNRATE,MCOILBRENTEU,HOUST,TOTALSA,RSXFS
14,2019-07-01,26805.859375,26890.640625,26616.210938,26717.429688,26717.429688,267670000.0,3.7,63.92,1212.0,17.442,457485.0
36,2019-08-01,26879.859375,27175.589844,26548.710938,26583.419922,26583.419922,386320000.0,3.7,59.04,1377.0,17.512,459938.0
78,2019-10-01,26962.539063,27046.210938,26562.220703,26573.039063,26573.039063,260110000.0,3.6,59.71,1340.0,17.046,459978.0
101,2019-11-01,27142.949219,27347.429688,27142.949219,27347.359375,27347.359375,270870000.0,3.5,63.21,1371.0,17.447,460760.0
204,2020-04-01,21227.380859,21487.240234,20784.429688,20943.509766,20943.509766,506680000.0,14.7,18.38,891.0,9.073,371585.0


In [83]:
# Elimina filas con outliers.
df = df[(np.abs(stats.zscore(df.drop(['DATE'], axis=1))) < 3).all(axis=1)]

# Muestra tamaño final después de eliminar outliers.
df.shape

(5, 12)

In [84]:
# Cambia columna fecha a datetime.datetime de Python.
df['DATE'] = df['DATE'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d"))

In [90]:
# Renombra columnas a títulos más amigables.
df.columns = ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'ADJ CLOSE', 'VOLUME', 'UNEMP %',
    'OIL PRICE','NEW HOMES','NEW CARS SOLD', 'RETAIL SALES']

# Previsualiza tabla final.
df.head(20)

,DATE,OPEN,HIGH,LOW,CLOSE,ADJ CLOSE,VOLUME,UNEMP %,OIL PRICE,NEW HOMES,NEW CARS SOLD,RETAIL SALES
14,2019-07-01,26805.859375,26890.640625,26616.210938,26717.429688,26717.429688,267670000.0,3.7,63.92,1212.0,17.442,457485.0
36,2019-08-01,26879.859375,27175.589844,26548.710938,26583.419922,26583.419922,386320000.0,3.7,59.04,1377.0,17.512,459938.0
78,2019-10-01,26962.539063,27046.210938,26562.220703,26573.039063,26573.039063,260110000.0,3.6,59.71,1340.0,17.046,459978.0
101,2019-11-01,27142.949219,27347.429688,27142.949219,27347.359375,27347.359375,270870000.0,3.5,63.21,1371.0,17.447,460760.0
204,2020-04-01,21227.380859,21487.240234,20784.429688,20943.509766,20943.509766,506680000.0,14.7,18.38,891.0,9.073,371585.0
